In [ ]:
# task
import os
import cv2
import numpy as np
import random

class DeGenerator:
    def __init__(self, image_dir, mask_dir, batch_size=4, res_size=(400, 400)):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.res_size = res_size
        
        
        self.image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')]
        self.mask_files = [os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.jpg')]
        
        if len(self.image_files) != len(self.mask_files):
            raise ValueError("Количество изображений и масок не совпадает")
        
        self.indices = list(range(len(self.image_files)))
        self.on_epoch_end()

    def on_epoch_end(self):
        
        random.shuffle(self.indices)

    def apply_augmentation(self, image, mask):
        if np.random.random() > 0.5:
            # 1. Поворот на случайный угол
            angle = np.random.uniform(-30, 30)
            M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1.0)
            image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
            mask = cv2.warpAffine(mask, M, (mask.shape[1], mask.shape[0]), flags=cv2.INTER_NEAREST)

        if np.random.random() > 0.5:
            # 2. Отражение
            if np.random.random() > 0.5:
                image = cv2.flip(image, 1)  
                mask = cv2.flip(mask, 1)
            if np.random.random() > 0.5:
                image = cv2.flip(image, 0)  
                mask = cv2.flip(mask, 0)
        
        if np.random.random() > 0.5:
            # 3. Вырезание куска
            h, w = image.shape[:2]
            crop_h = int(h * np.random.uniform(0.7, 1.0))
            crop_w = int(w * np.random.uniform(0.7, 1.0))
            top = np.random.randint(0, h - crop_h)
            left = np.random.randint(0, w - crop_w)
            image = image[top:top + crop_h, left:left + crop_w]
            mask = mask[top:top + crop_h, left:left + crop_w]
        
        if np.random.random() > 0.5:
            # 4. Хентай
            image = cv2.GaussianBlur(image, (11, 11), 0)  

        
        image = cv2.resize(image, self.res_size)
        mask = cv2.resize(mask, self.res_size, interpolation=cv2.INTER_NEAREST)

        return image, mask

    def apply_mask(self, image, mask):
        
        mask_color = (0, 0, 255)  
        alpha = 0.5  
        mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)  
        mask_normalized = mask.astype(float) / 255.0  

        
        for c in range(0, 3):
            image[:, :, c] = (1 - mask_normalized * alpha) * image[:, :, c] + \
                           (mask_normalized * alpha) * mask_color[c]

        return image

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_results = []

        for i in batch_indices:
            img_path = self.image_files[i]
            mask_path = self.mask_files[i]

            image = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if image is None or mask is None:
                raise ValueError(f"Не удалось загрузить {img_path} или {mask_path}")

            aug_image, aug_mask = self.apply_augmentation(image, mask)
            result = self.apply_mask(aug_image, aug_mask)

            
            result = result / 255.0

            batch_results.append(result)

        return np.array(batch_results)

    def __len__(self):
        return len(self.indices) // self.batch_size

def main():
    image_dir = "nails_segmentation/images"
    mask_dir = "nails_segmentation/labels"
    generator = DeGenerator(image_dir, mask_dir, batch_size=4, res_size=(400, 400))
    
    
    for i in range(len(generator)):
        batch = generator[i]
    
        cv2.imshow('res', batch[0])
        if cv2.waitKey(0) & 0xFF == 27:  
            break

    cv2.destroyAllWindows()


main()

In [4]:
# ghosts
import cv2
import numpy as np
from sklearn.cluster import DBSCAN


def find_unique_points(image, original_image):
    im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    base_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    mir_im = im_gray.copy()
    mir_im = cv2.flip(mir_im, 1)

    sift = cv2.SIFT_create()

    keypoints_1, descriptors_1 = sift.detectAndCompute(base_gray, None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(im_gray, None)
    keypoints_3, descriptors_3 = sift.detectAndCompute(mir_im, None)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True) 
    
    matches1 = bf.match(descriptors_1, descriptors_2)
    matches2 = bf.match(descriptors_1, descriptors_3)

   
    matches1 = sorted(matches1, key = lambda x:x.distance)
    matches2 = sorted(matches2, key = lambda x:x.distance)


    src_pts_1 = np.float32([keypoints_1[m.queryIdx].pt for m in matches1]).reshape(-1, 2)
    dst_pts_1 = np.float32([keypoints_2[m.trainIdx].pt for m in matches1]).reshape(-1, 2)

    src_pts_2 = np.float32([keypoints_1[m.queryIdx].pt for m in matches2]).reshape(-1, 2)
    dst_pts_2 = np.float32([keypoints_3[m.trainIdx].pt for m in matches2]).reshape(-1, 2)

    all_src_pts = np.vstack([src_pts_1, src_pts_2])
    all_dst_pts = np.vstack([dst_pts_1, dst_pts_2])

    if len(all_src_pts) < 4 or len(all_dst_pts) < 4:
        print("Недостаточно соответствий для вычисления гомографии")
        return original_image.copy(), original_image.copy()

    
    dbscan = DBSCAN(eps= 100, min_samples=4)
    clusters = dbscan.fit_predict(all_src_pts)

    result = original_image.copy()

    for cluster_id in np.unique(clusters):
        if cluster_id == -1: continue  
        cluster_mask = clusters == cluster_id
        src_cluster = all_src_pts[cluster_mask]
        dst_cluster = all_dst_pts[cluster_mask]

        if len(src_cluster) < 18: continue  

        H, mask = cv2.findHomography(src_cluster, dst_cluster, cv2.RANSAC, 5.0)

        
        if H is None or H.shape[0] < 3 or H.shape[1] < 3:
            print(f"Некорректная гомография для кластера {cluster_id}")
            continue

        h, w = image.shape[:2]
        corners = np.float32([[0, 0], [w - 1, 0], [w - 1, h - 1], [0, h-1]]).reshape(-1, 1, 2)
        H_inv = np.linalg.inv(H)
        transformed_corners = cv2.perspectiveTransform(corners, H_inv)

        top_left = (int(min(transformed_corners[:, 0, 0])), int(min(transformed_corners[:, 0, 1])))
        bottom_right = (int(max(transformed_corners[:, 0, 0])), int(max(transformed_corners[:, 0, 1])))

        cv2.rectangle(result, top_left, bottom_right, (0, 255, 0), 5)

    return result

scene = cv2.imread('ghosts/lab7.png')  
candy_ghost = cv2.imread('ghosts/candy_ghost.png') 
pampkin_ghost = cv2.imread('ghosts/pampkin_ghost.png')
scary_ghost = cv2.imread('ghosts/scary_ghost.png')

result = scene.copy()
result = find_unique_points(candy_ghost, result)
result = find_unique_points(pampkin_ghost, result)
result = find_unique_points(scary_ghost, result)

x, y = result.shape[:2]
result = cv2.resize(result, (y//2, x//2))



cv2.imshow('res', result)

cv2.waitKey(0)
cv2.destroyAllWindows()